#### Hasta este punto tenemos nuestras tablas procesadas (movies, ratings y tags) de forma correcta. El cliente nos ha solicitado apoyar al departamento de Marketing a realizar algunas consultas y a generar una única tabla y realizar algunos ajustes a la tabla final antes de poder almacenarla.

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de la sesión 07

In [1]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.ipynb

In [2]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession, DataFrame, Column
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_8") \
    .getOrCreate()

# Carga de tablas requeridas
root_path = "../resources/data/tmp/parquet/"
names_list = ["07/movies", "07/ratings", "07/tags_p2"]
df_dict = read_tmp_df(names_list)

movies_df = df_dict["07/movies"]
ratings_df = df_dict["07/ratings"]
tags_df = df_dict["07/tags_p2"]

movies_df.show(1, False)
ratings_df.show(1, False)
tags_df.show(1, False)

+--------+----------------+-------------------------------------------------+----+
|movie_id|title           |genres                                           |year|
+--------+----------------+-------------------------------------------------+----+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1995|
+--------+----------------+-------------------------------------------------+----+
only showing top 1 row

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|min_time_rating    |
+--------+----------+-------------+------------+-------------------+
|3298    |3.5       |0.88         |5743        |2000-02-17 01:59:39|
+--------+----------+-------------+------------+-------------------+
only showing top 1 row

+--------+----------------------------+-------------------+
|movie_id|tag_count                   |min_time_tag       |
+--------+----------------------------+-------------------+
|100048  |[{

#### En la siguiente imagen se muestra una representación a traves del diagrama de Venn sobre cada tabla (movies_df, ratings_df y tags_df) la cual fue contruida por el departamento de Marketing, el problema es que no saben la cantidad de datos de cada conjunto.
##### El cliente nos han solicitado obtener la cantidad de registros de cada conjunto de datos representado por las siguientes letras:
- ##### A: Registros de movies_df que no tiene filas coincidentes con las tablas ratings_df y tags_df
- ##### B: Registros de ratings_df que no tiene filas coincidentes con las tablas movies_df y tags_df
- ##### C: Registros de tags_df que no tiene filas coincidentes con las tablas movies_df y ratings_df
- ##### D: Registros de movies_df y ratings_df que no tiene filas coincidentes con la tabla tags_df
- ##### E: Registros de movies_df y tags_df que no tiene filas coincidentes con la tabla ratings_df
- ##### F: Registros de ratings_df y tags_df que no tiene filas coincidentes con la tabla movies_df
- ##### G: Registros que contiene datos coincidentes en las tablas movies_df, ratings_df y tags_df
-  Una tabla tiene registros coincidentes con otra cuando comparten el mismo valor en la columna "movie_id"

![title](../resources/img/Tablas_conjuntos.png)

#### Actividad 1:
##### TO DO ->    Obtener los conjuntos de datos listados en el diagrama de Venn con operaciones de tipo join, con la finalidad de ahorrar recursos el cliente nos ha solicitado que en la salida de cada transformación el dataframe resultante contenga únicamente la columna "movie_id" (basta con utilizar únicante una trasnsformación select al inicio de las operaciones join y utilizar únicamente joins del tipo left_semi y left_anti).
- ##### Los dataframes resultantes se almacenarán en las siguientes variables:
    - ##### A_df -> Conjunto A
    - ##### B_df -> Conjunto B
    - ##### C_df -> Conjunto C
    - ##### D_df -> Conjunto D
    - ##### E_df -> Conjunto E
    - ##### F_df -> Conjunto F
    - ##### G_df -> Conjunto G

In [3]:
## TU CODIGO VA EN ESTA CELDA:

#Importaciones extra 
import pyspark.sql.functions as f

# A: Registros de movies_df que no tienen coincidencias en ratings_df ni tags_df
A_df = movies_df.select("movie_id").join(ratings_df, "movie_id", "left_anti").join(tags_df, "movie_id", "left_anti")

# B: Registros de ratings_df sin coincidencias en movies_df ni tags_df
B_df = ratings_df.select("movie_id").join(movies_df, "movie_id", "left_anti").join(tags_df, "movie_id", "left_anti")

# C: Registros de tags_df sin coincidencias en movies_df ni ratings_df
C_df = tags_df.select("movie_id").join(movies_df, "movie_id", "left_anti").join(ratings_df, "movie_id", "left_anti")

# D: Registros de movies_df y ratings_df sin coincidencias en tags_df
D_df = movies_df.select("movie_id").join(ratings_df, "movie_id", "left_semi").join(tags_df, "movie_id", "left_anti")

# E: Registros de movies_df y tags_df sin coincidencias en ratings_df
E_df = movies_df.select("movie_id").join(tags_df, "movie_id", "left_semi").join(ratings_df, "movie_id", "left_anti")

# F: Registros de ratings_df y tags_df sin coincidencias en movies_df
F_df = ratings_df.select("movie_id").join(tags_df, "movie_id", "left_semi").join(movies_df, "movie_id", "left_anti")

# G: Registros que contienen datos coincidentes en movies_df, ratings_df y tags_df
G_df = movies_df.select("movie_id").join(ratings_df, "movie_id", "left_semi").join(tags_df, "movie_id", "left_semi")

# Imprimir resultados
A_df.show()
B_df.show()
C_df.show()
D_df.show()
E_df.show()
F_df.show()
G_df.show()

+--------+
|movie_id|
+--------+
+--------+

+--------+
|movie_id|
+--------+
|  134340|
|  171705|
|  135819|
|  232045|
|  196145|
|  190483|
|  194374|
|  151379|
|  157833|
|  172631|
|  278602|
|  155529|
|  236969|
|  207387|
|  174113|
|  155106|
|  132247|
|  215753|
|  170671|
|  125640|
+--------+
only showing top 20 rows

+--------+
|movie_id|
+--------+
|  123755|
|  128295|
|  129869|
|  130776|
|  133433|
|  135354|
|  145436|
|  146495|
|  146616|
|  147676|
|  149270|
|  149896|
|  151115|
|  151943|
|  153470|
|  155838|
|  156268|
|  156525|
|  157717|
|  161252|
+--------+
only showing top 20 rows

+--------+
|movie_id|
+--------+
|      51|
|      56|
|      84|
|      90|
|     108|
|     139|
|     142|
|     192|
|     200|
|     284|
|     287|
|     310|
|     401|
|     406|
|     447|
|     463|
|     476|
|     579|
|     603|
|     604|
+--------+
only showing top 20 rows

+--------+
|movie_id|
+--------+
|   52696|
|   58209|
|   69864|
|   70312|
|   7135

In [4]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
B_df.show(1)
"""
Ejemplo de salida de cada dataframe (desde A hasta G):
+--------+
|movie_id|
+--------+
|  179995|
+--------+
only showing top 1 row
"""

+--------+
|movie_id|
+--------+
|  134340|
+--------+
only showing top 1 row



'\nEjemplo de salida de cada dataframe (desde A hasta G):\n+--------+\n|movie_id|\n+--------+\n|  179995|\n+--------+\nonly showing top 1 row\n'

In [5]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

assert A_df.count() == 0
assert B_df.count() == 4270
assert C_df.count() == 539
assert D_df.count() == 28815
assert E_df.count() == 2759
assert F_df.count() == 2251
assert G_df.count() == 47903

assert len(A_df.columns) == 1
assert len(B_df.columns) == 1
assert len(C_df.columns) == 1
assert len(D_df.columns) == 1
assert len(E_df.columns) == 1
assert len(F_df.columns) == 1
assert len(G_df.columns) == 1

assert "movie_id" in A_df.columns
assert "movie_id" in B_df.columns
assert "movie_id" in C_df.columns
assert "movie_id" in D_df.columns
assert "movie_id" in E_df.columns
assert "movie_id" in F_df.columns
assert "movie_id" in G_df.columns

#### Actividad 2:
##### TO DO ->    Algunos administradores de base de datos no comprenden el uso de los joins de tipo left_semi y left_anti, por lo tanto el cliente ha solicitado que tambien se realicen las transformaciones con cualquiera de los siguientes tipos de join: left, right, inner, outer. Podrías utilizar la transformación filter/where en algunos casos.
##### No existe alguna reestricción de qué columnas contiene el dataframe de salida.
- ##### Los dataframes resultantes se almacenarán en las siguientes variables:
    - ##### A_df -> Conjunto A
    - ##### B_df -> Conjunto B
    - ##### C_df -> Conjunto C
    - ##### D_df -> Conjunto D
    - ##### E_df -> Conjunto E
    - ##### F_df -> Conjunto F
    - ##### G_df -> Conjunto G

In [6]:
## TU CODIGO VA EN ESTA CELDA:
# Importaciones extra
import pyspark.sql.functions as f

# A: Registros de movies_df que no tienen coincidencias en ratings_df ni tags_df
A_df = movies_df \
    .join(ratings_df, ["movie_id"], "left") \
    .join(tags_df, ["movie_id"], "left") \
    .filter(f.col("count_rating").isNull() & f.col("tag_count").isNull())

# B: Registros de ratings_df sin coincidencias en movies_df ni tags_df
B_df = ratings_df \
    .join(movies_df, ["movie_id"], "left") \
    .join(tags_df, ["movie_id"], "left") \
    .filter(f.col("title").isNull() & f.col("tag_count").isNull())

# C: Registros de tags_df sin coincidencias en movies_df ni ratings_df
C_df = tags_df \
    .join(movies_df, ["movie_id"], "left") \
    .join(ratings_df, ["movie_id"], "left") \
    .filter(f.col("title").isNull() & f.col("count_rating").isNull())

# D: Registros de movies_df y ratings_df sin coincidencias en tags_df
D_df = movies_df \
    .join(ratings_df, ["movie_id"], "inner") \
    .join(tags_df, ["movie_id"], "left") \
    .filter(f.col("tag_count").isNull())

# E: Registros de movies_df y tags_df sin coincidencias en ratings_df
E_df = movies_df \
    .join(tags_df, ["movie_id"], "inner") \
    .join(ratings_df, ["movie_id"], "left") \
    .filter(f.col("count_rating").isNull())

# F: Registros de ratings_df y tags_df sin coincidencias en movies_df
F_df = tags_df \
    .join(ratings_df, ["movie_id"], "inner") \
    .join(movies_df, ["movie_id"], "left") \
    .filter(f.col("title").isNull())

# G: Registros que contienen datos coincidentes en movies_df, ratings_df y tags_df
G_df = movies_df \
    .join(ratings_df, ["movie_id"], "inner") \
    .join(tags_df, ["movie_id"], "inner")

# Imprimir resultados
A_df.show()
B_df.show()
C_df.show()
D_df.show()
E_df.show()
F_df.show()
G_df.show()

+--------+-----+------+----+----------+-------------+------------+---------------+---------+------------+
|movie_id|title|genres|year|avg_rating|stddev_rating|count_rating|min_time_rating|tag_count|min_time_tag|
+--------+-----+------+----+----------+-------------+------------+---------------+---------+------------+
+--------+-----+------+----+----------+-------------+------------+---------------+---------+------------+

+--------+----------+-------------+------------+-------------------+-----+------+----+---------+------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|title|genres|year|tag_count|min_time_tag|
+--------+----------+-------------+------------+-------------------+-----+------+----+---------+------------+
|  134340|      3.36|         1.03|           7|2015-05-30 19:18:26| null|  null|null|     null|        null|
|  171705|      3.93|         1.21|          38|2017-05-03 01:05:48| null|  null|null|     null|        null|
|  135819|       3.0|    

In [7]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

assert A_df.count() == 0
assert B_df.count() == 4270
assert C_df.count() == 539
assert D_df.count() == 28815
assert E_df.count() == 2759
assert F_df.count() == 2251
assert G_df.count() == 47903

In [8]:
## CELDA ADICIONAL PARA PRUEBAS 

# DataFrame y tipos de datos de movies_id
movies_df.show()
movies_df.printSchema()

# DataFrame y tipos de datos de ratings_df
ratings_df.show()
ratings_df.printSchema()

#DataFrame y tipos de datos de tags_df
tags_df.show()
tags_df.printSchema()

+--------+--------------------+--------------------+----+
|movie_id|               title|              genres|year|
+--------+--------------------+--------------------+----+
|       1|    Toy Story (1995)|[Adventure, Anima...|1995|
|       2|      Jumanji (1995)|[Adventure, Child...|1995|
|       3|Grumpier Old Men ...|   [Comedy, Romance]|1995|
|       4|Waiting to Exhale...|[Comedy, Drama, R...|1995|
|       5|Father of the Bri...|            [Comedy]|1995|
|       6|         Heat (1995)|[Action, Crime, T...|1995|
|       7|      Sabrina (1995)|   [Comedy, Romance]|1995|
|       8| Tom and Huck (1995)|[Adventure, Child...|1995|
|       9| Sudden Death (1995)|            [Action]|1995|
|      10|    GoldenEye (1995)|[Action, Adventur...|1995|
|      11|American Presiden...|[Comedy, Drama, R...|1995|
|      12|Dracula: Dead and...|    [Comedy, Horror]|1995|
|      13|        Balto (1995)|[Adventure, Anima...|1995|
|      14|        Nixon (1995)|             [Drama]|1995|
|      15|Cutt

#### Actividad 3:
##### TO DO ->    Con operaciones join genera un dataframe que contenga la union de todos los conjuntos (desde el A hasta el G) sin repetir registros.

In [9]:
## TU CODIGO VA EN ESTA CELDA:

# Union de los conjuntos A - G
universe_df = movies_df.join(ratings_df, ["movie_id"], "outer").join(tags_df, ["movie_id"], "outer")
universe_df.show(truncate=False)

+--------+-----------------------------------------------+-----------------------------------+----+----------+-------------+------------+-------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
universe_df.show(1)
"""
Ejemplo de salida:
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|               title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|  100062|My Way (Mai Wei) ...|[Action, Drama, War]|2011|      3.63|         0.83|          64|2014-03-11 21:23:33|[{PRESS-GANGED, 1...|2018-05-26 16:40:54|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 1 row
"""

+--------+--------------------+--------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|               title|  genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+--------------------+--------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|  100070|Punching the Clow...|[Comedy]|2009|      3.54|         0.89|          13|2013-01-24 11:24:50|[{GOOD HUMOUR, 1}...|2017-05-19 17:17:36|
+--------+--------------------+--------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 1 row



'\nEjemplo de salida:\n+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\n|movie_id|               title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|\n+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\n|  100062|My Way (Mai Wei) ...|[Action, Drama, War]|2011|      3.63|         0.83|          64|2014-03-11 21:23:33|[{PRESS-GANGED, 1...|2018-05-26 16:40:54|\n+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\nonly showing top 1 row\n'

In [11]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

test_value = [Row(
    movie_id='100062', 
    title='My Way (Mai Wei) (2011)', 
    genres=['Action', 'Drama', 'War'], 
    year=2011, 
    avg_rating=3.63, 
    stddev_rating=0.83, 
    count_rating=64, 
    min_time_rating=datetime.datetime(2014, 3, 11, 21, 23, 33), 
    tag_count=[
        Row(tag='PRESS-GANGED', count=1),
        Row(tag='WORLD WAR II', count=1),
        Row(tag='WAR', count=1),
        Row(tag='FATE', count=1)],
    min_time_tag=datetime.datetime(2018, 5, 26, 16, 40, 54))]

expected_columns = ["movie_id", "title", "genres", "year",
                    "avg_rating", "stddev_rating", "count_rating",
                    "min_time_rating", "tag_count", "min_time_tag"]

assert universe_df.count() == 86537
assert set(universe_df.columns) - set(expected_columns) == set()
assert set(expected_columns) - set(universe_df.columns) == set()
assert universe_df \
    .select(*expected_columns) \
    .filter(f.col("movie_id") == "100062") \
    .collect() == test_value

#### Actividad 4:
##### TO DO ->    La estructura del dataframe final de acuerdo al análisis realizado por marketing es el dado por la union del conjunto de datos: A, D, E y G. Realiza este proceso y almacena el dataframe resultante en la variable "final_df"
##### 

In [12]:
# CELDA ADICIONAL 

# Conteo de movies_df
movies_df.count()

79477

In [13]:
# CELDA ADICIONAL 

# Conteo de ratings_df
ratings_df.count()

83239

In [14]:
# CELDA ADICIONAL 

# Conteo de tags_df
tags_df.count()

53452

In [15]:
## TU CODIGO VA EN ESTA CELDA:

# Importaciones extra
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, DoubleType, LongType, IntegerType

# Casteos y selección de las columnas para "movies_df"
movies_select = movies_df.select(
    col("movie_id").cast(StringType()).alias("movie_id"),
    col("title"),
    col("genres"),
    col("year").cast(IntegerType()).alias("year")
)

# Casteo y selección de las columnas para "ratings_df"
ratings_select = ratings_df.select(
    col("movie_id").cast(StringType()).alias("movie_id"),
    col("avg_rating").cast(DoubleType()).alias("avg_rating"),
    col("stddev_rating").cast(DoubleType()).alias("stddev_rating"),
    col("count_rating").cast(LongType()).alias("count_rating"),
    col("min_time_rating")
)

# Casteo y selección de las columnas para "tags_df"
tags_select = tags_df.select(
    col("movie_id").cast(StringType()).alias("movie_id"),
    col("tag_count"),
    col("min_time_tag")
)

# Realiza las uniones de los DataFrames seleccionados utilizando join outer
joined_df = movies_select.join(ratings_select, ["movie_id"], "left")
final_df = joined_df.join(tags_select, ["movie_id"], "left")

# Muestra el resultado
final_df.show()
final_df.count()

+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|               title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+--------------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|       1|    Toy Story (1995)|[Adventure, Anima...|1995|      3.89|         0.93|       76813|1996-01-28 18:00:00|[{BEST OF ROTTEN ...|2006-01-12 19:19:35|
|       2|      Jumanji (1995)|[Adventure, Child...|1995|      3.28|         0.96|       30209|1996-01-28 18:00:00|[{KIDS, 1}, {DRIV...|2006-01-13 14:14:30|
|       3|Grumpier Old Men ...|   [Comedy, Romance]|1995|      3.17|         1.02|       15820|1996-01-29 17:54:45|[{OLD MEN, 1}, {O...|2006-01-16 00:28:45|
|       4|Waiting to Exhale...|[Comedy, Drama, R...|1995| 

79477

In [16]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
final_df.show(1)
"""
Ejemplo de salida:
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|           title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|      3.89|         0.93|       76813|1996-01-28 18:00:00|[{TIME TRAVEL, 11...|2006-01-12 19:19:35|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 1 row
"""

+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|movie_id|           title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|      3.89|         0.93|       76813|1996-01-28 18:00:00|[{BEST OF ROTTEN ...|2006-01-12 19:19:35|
+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+
only showing top 1 row



'\nEjemplo de salida:\n+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\n|movie_id|           title|              genres|year|avg_rating|stddev_rating|count_rating|    min_time_rating|           tag_count|       min_time_tag|\n+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\n|       1|Toy Story (1995)|[Adventure, Anima...|1995|      3.89|         0.93|       76813|1996-01-28 18:00:00|[{TIME TRAVEL, 11...|2006-01-12 19:19:35|\n+--------+----------------+--------------------+----+----------+-------------+------------+-------------------+--------------------+-------------------+\nonly showing top 1 row\n'

In [17]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

test_value = [Row(
    movie_id='100062', 
    title='My Way (Mai Wei) (2011)', 
    genres=['Action', 'Drama', 'War'], 
    year=2011, 
    avg_rating=3.63, 
    stddev_rating=0.83, 
    count_rating=64, 
    min_time_rating=datetime.datetime(2014, 3, 11, 21, 23, 33), 
    tag_count=[
        Row(tag='PRESS-GANGED', count=1),
        Row(tag='WORLD WAR II', count=1),
        Row(tag='WAR', count=1),
        Row(tag='FATE', count=1)],
    min_time_tag=datetime.datetime(2018, 5, 26, 16, 40, 54))]

expected_columns = ["movie_id", "title", "genres", "year",
                    "avg_rating", "stddev_rating", "count_rating",
                    "min_time_rating", "tag_count", "min_time_tag"]

assert final_df.count() == 79477

assert set(final_df.columns) - set(expected_columns) == set()
assert set(expected_columns) - set(final_df.columns) == set()
assert final_df \
    .select(*expected_columns) \
    .filter(f.col("movie_id") == "100062") \
    .collect() == test_value

#### Actividad 5:
##### TO DO ->    El cliente nos ha solicitado llenar los valores "null" de la columna "year", para esto nos ha pedido seguir la siguiente regla:
- ##### Si la columna "year" es null, colocar el año menor de las columnas "min_time_rating" y "min_time_tag"
- ##### Si la columna "year" es null y si la columna "min_time_rating" es null colocar el año de la columna "min_time_tag".
- ##### Si la columna "year" es null y si la columna "min_time_tag" es null colocar el año de la columna "min_time_rating".
- ##### En cualquier otro caso mantener el valor entero 1970
##### Adicional nos ha solicitado generar una columna llamada "year_type" con los siguientes valores:
- ##### Si la columna "year" venia con un valor distinto a null, asignar el valor "YO"
- ##### Si a la columna "year" tenia valor null y las columnas "min_time_tag" y "min_time_rating" tienen el mismo año asignar "YRT"
- ##### Si a la columna "year" tenia valor null y ((el año de la columna "min_time_tag" es menor al año de "min_time_rating") o (la columna  min_time_rating es null)) asignar "YT"
- ##### Si a la columna "year" tenia valor null y ((el año de la columna "min_time_rating" es menor al año de "min_time_tag") o (la columna  min_time_tag es null)) asignar "YR"
- ##### En cualquier otro caso mantener "YU"
##### Al finalizar este proceso eliminar las columnas "min_time_rating" y "min_time_tag"
- Para resolver estos ejercicios podrías utilizar la función year -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.year.html#pyspark.sql.functions.year
##### Este proceso se desarrollará en un método con la firma: def get_last_movies_df(df: DataFrame) -> DataFrame
##### La estructura del dataframe de salida será:
* |-- movie_id: string
* |-- title: string
* |-- avg_rating: double
* |-- count_rating: long
* |-- stddev_rating: double
* |-- genres: array
* |--- |-- element: string
* |-- tag_count: array
* |--- |-- element: struct
* |--- |--- |tag: string
* |--- |--- |count: long
* |-- year: integer
* |-- year_type: string
##### NO UTILIZAR withColumn NI withColumnRenamed 

In [34]:
## TU CODIGO VA EN ESTA CELDA:
# Importaciones extra
from pyspark.sql import DataFrame
from pyspark.sql.functions import year, when, col, lit, array_sort 

# Función principal para procesar el DataFrame (aplicar a final_df)
def get_last_movies_df(df: DataFrame) -> DataFrame:
    result_df = df.select(
        col("movie_id"),
        col("title"),
        col("avg_rating"),
        col("count_rating"),
        col("stddev_rating"),
        col("genres"),
        array_sort(col("tag_count")).alias("tag_count"),
        when(col("year").isNotNull(), "YO") \
           .when((year("min_time_rating")<year("min_time_tag"))|(col("min_time_tag").isNull()), "YR") \
           .when((year("min_time_tag")<year("min_time_rating"))|(col("min_time_rating").isNull()), "YT") \
           .when(year("min_time_tag")==year("min_time_rating"), "YRT") \
           .otherwise(lit("YU")).alias("year_type"),      
        when(col("year").isNotNull(), col("year")) \
           .when((year("min_time_rating")<=year("min_time_tag"))|(col("min_time_tag").isNull()), year(col("min_time_rating"))) \
           .when((year("min_time_tag")<year("min_time_rating"))|(col("min_time_rating").isNull()), year(col("min_time_tag"))) \
           .otherwise(lit(1970)).alias("year")
    )
    return result_df
    
# Llamar a la función con tu DataFrame de entrada
result_df = get_last_movies_df(final_df)

# Mostrar el resultado
result_df.show()

# Esquema de datos
result_df.printSchema()

+--------+--------------------+----------+------------+-------------+--------------------+--------------------+---------+----+
|movie_id|               title|avg_rating|count_rating|stddev_rating|              genres|           tag_count|year_type|year|
+--------+--------------------+----------+------------+-------------+--------------------+--------------------+---------+----+
|       1|    Toy Story (1995)|      3.89|       76813|         0.93|[Adventure, Anima...|[{1990S, 1}, {200...|       YO|1995|
|       2|      Jumanji (1995)|      3.28|       30209|         0.96|[Adventure, Child...|[{1860S, 1}, {196...|       YO|1995|
|       3|Grumpier Old Men ...|      3.17|       15820|         1.02|   [Comedy, Romance]|[{ANN MARGARET, 1...|       YO|1995|
|       4|Waiting to Exhale...|      2.87|        3028|         1.11|[Comedy, Drama, R...|[{BASED ON NOVEL ...|       YO|1995|
|       5|Father of the Bri...|      3.08|       15801|          1.0|            [Comedy]|[{4TH WALL, 1}, {...|

In [31]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

movies_df = get_last_movies_df(final_df)
movies_df.show(1)
"""
Ejemplo de salida esperada (el orden de la columnas podría ser distinto):
+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
|movie_id|           title|avg_rating|count_rating|stddev_rating|              genres|           tag_count|year|year_type|
+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
|       1|Toy Story (1995)|      3.89|       76813|         0.93|[Adventure, Anima...|[{TIME TRAVEL, 11...|1995|       YO|
+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
only showing top 1 row
"""

+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
|movie_id|           title|avg_rating|count_rating|stddev_rating|              genres|           tag_count|year|year_type|
+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
|       1|Toy Story (1995)|      3.89|       76813|         0.93|[Adventure, Anima...|[{1990S, 1}, {200...|1995|       YO|
+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+
only showing top 1 row



'\nEjemplo de salida esperada (el orden de la columnas podría ser distinto):\n+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+\n|movie_id|           title|avg_rating|count_rating|stddev_rating|              genres|           tag_count|year|year_type|\n+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+\n|       1|Toy Story (1995)|      3.89|       76813|         0.93|[Adventure, Anima...|[{TIME TRAVEL, 11...|1995|       YO|\n+--------+----------------+----------+------------+-------------+--------------------+--------------------+----+---------+\nonly showing top 1 row\n'

In [35]:
# CELDA ADICIONAL
movies_df \
    .select(*expected_columns) \
    .filter(f.col("movie_id") == "179479").show(1, False)

movies_df.groupBy(f.col("year_type")).count().orderBy(f.col("count").desc()).show()

+--------+----------------------------------------------+----------+------------+-------------+-------------+-------------------------------------------------------------------------------------------+----+---------+
|movie_id|title                                         |avg_rating|count_rating|stddev_rating|genres       |tag_count                                                                                  |year|year_type|
+--------+----------------------------------------------+----------+------------+-------------+-------------+-------------------------------------------------------------------------------------------+----+---------+
|179479  |Samadhi Part 1: Maya, the Illusion of the Self|4.19      |8           |0.75         |[Documentary]|[{EASTERN PHILOSOPHY, 1}, {MEDITATION, 1}, {METAPHYSICAL, 1}, {NEW AGE, 1}, {SPIRITUAL, 1}]|2017|YT       |
+--------+----------------------------------------------+----------+------------+-------------+-------------+-----------------------

In [28]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

test_value = [Row(
    movie_id='179479',
    title='Samadhi Part 1: Maya, the Illusion of the Self',
    avg_rating=4.19,
    count_rating=8,
    stddev_rating=0.75,
    genres=['Documentary'],
    tag_count=[Row(tag='EASTERN PHILOSOPHY', count=1),
               Row(tag='MEDITATION', count=1),
               Row(tag='METAPHYSICAL', count=1),
               Row(tag='NEW AGE', count=1),
               Row(tag='SPIRITUAL', count=1)],
    year=2017,
    year_type='YT')]

expected_count_by_year_type = [
    Row(year_type='YO', count=79235),
    Row(year_type='YR', count=159),
    Row(year_type='YRT', count=55),
    Row(year_type='YT', count=28)]

expected_columns = ["movie_id","title","avg_rating","count_rating","stddev_rating","genres","tag_count","year","year_type"]

expected_schema = 'movie_id STRING,title STRING,avg_rating DOUBLE,count_rating BIGINT,stddev_rating DOUBLE,genres ARRAY<STRING>,tag_count ARRAY<STRUCT<tag: STRING, count: BIGINT>>,year INT,year_type STRING NOT NULL'

assert set(movies_df.columns) - set(expected_columns) == set()
assert set(expected_columns) - set(movies_df.columns) == set()
assert movies_df.filter(f.col("year").isNull()).count() == 0
assert movies_df \
    .select(*expected_columns) \
    .filter(f.col("movie_id") == "179479") \
    .collect() == test_value
assert schema_to_ddl(movies_df.select(*expected_columns)) == expected_schema
assert movies_df.groupBy(f.col("year_type")).count().orderBy(f.col("count").desc()).collect() == expected_count_by_year_type

AssertionError: 

In [ ]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(movies_df, "08/movies")]

write_tmp_df(dfs)